### Topic Modeling

In [1]:
#open txt file
with open ('/Users/huopeiyang/Desktop/Calvino, Italo - Invisible Cities (transl. William Weaver) (2012) - libgen.li.txt') as in_file:
    text=in_file.read()


In [2]:
import re
regex = r"\n[A-Z]+[\s&]*[A-Z\s]+\s*[0-9]\s*\n\n(.+?)\n\n\n"
content_each=re.findall(regex, text)

In [3]:
import pandas as pd 
from bertopic import BERTopic 
from sentence_transformers import SentenceTransformer, util 
from umap import UMAP 
from hdbscan import HDBSCAN 
from sklearn.feature_extraction.text import CountVectorizer 
from bertopic.vectorizers import ClassTfidfTransformer 
import numpy as np 
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

/Users/huopeiyang/anaconda3/envs/tf/lib/python3.9/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/huopeiyang/anaconda3/envs/tf/lib/python3.9/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/huopeiyang/anaconda3/envs/tf/lib/python3.9/site-packages/umap/d

In [4]:
import nltk
nltk.download('punkt')  # This is the Punkt sentence tokenizer.

# This will contain each sentence as a separate document.
sentences_as_documents_essay = []

for document in content_each:
    # Tokenize the current document into sentences.
    sentences = nltk.sent_tokenize(document)
    
    # Add the sentences to our new list.
    sentences_as_documents_essay.extend(sentences)

# Now, sentences_as_documents contains each sentence as a separate document.


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/huopeiyang/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
docs=sentences_as_documents_essay

In [6]:
from bertopic.representation import KeyBERTInspired
from bertopic.vectorizers import ClassTfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

In [38]:
custom_stopwords=['berenices','lares','penates','inhabitants','city','cities','diomira', 'isidora', 'dorothea', 'zaira', 'anastasia', 'tamara', 'zora', 'despina', 'zirma', 'isaura', 'maurilia', 'fedora', 'zoe', 'zenobia', 'euphemia', 'zobeide', 'hypatia', 'armilla', 'chloe', 'valdrada', 'olivia', 'sophronia', 'eutropia', 'zemrude', 'aglaura', 'octavia', 'ersilia', 'baucis', 'leandra', 'melania', 'esmeralda', 'phyllis', 'pyrrha', 'adelma', 'eudoxia', 'moriana', 'clarice', 'eusapia', 'beersheba', 'leonia', 'irene', 'argia', 'thekla', 'trude', 'olinda', 'laudomia', 'perinthia', 'procopia', 'raissa', 'andria', 'cecilia', 'marozia', 'penthesilea', 'theodora', 'berenice']
all_stopwords=list(ENGLISH_STOP_WORDS)+ custom_stopwords
vectorizer_model = CountVectorizer(stop_words= all_stopwords ) 

ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)
representation_model = KeyBERTInspired()
sentence_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = sentence_model.encode(docs, show_progress_bar=True)
umap_model = UMAP(n_neighbors=15, n_components=5, 
                  min_dist=0.0, metric='cosine', random_state=41)

topic_model = BERTopic(vectorizer_model=vectorizer_model,
                      ctfidf_model=ctfidf_model,
                      representation_model=representation_model,
                      embedding_model=sentence_model,
                      umap_model=umap_model)
topics, probs = topic_model.fit_transform(docs)
topic_model.get_topic_info()

Batches:   0%|          | 0/19 [00:00<?, ?it/s]

Topic  Count                                            Name  \
0     -1    293               -1_outskirts_houses_streets_place   
1      0     72            0_palaces_construction_floors_street   
2      1     46     1_necropolis_corpses_inhabitant_generations   
3      2     44              2_curtain_caravan_camel_glittering   
4      3     30                3_rodents_plagues_rats_invasions   
5      4     29  4_astronomers_constellations_nebula_telescopes   
6      5     25    5_realizing_language_existence_understanding   
7      6     22      6_staircases_travellers_streets_encounters   
8      7     18            7_terraces_fountains_roofs_windmills   
9      8     12                8_goats_pasture_grazing_goatherd   

                                      Representation  \
0  [outskirts, houses, streets, place, canals, me...   
1  [palaces, construction, floors, street, justic...   
2  [necropolis, corpses, inhabitant, generations,...   
3  [curtain, caravan, camel, glittering, window, ...   
4  [rodents, plagues, rats, invasions, lairs, fau...   
5  [astronomers, constellations, nebula, telescop...   
6  [realizing, language, existence, understanding...   
7  [staircases, travellers, streets, encounters, ...   
8  [terraces, fountains, roofs, windmills, pyrami...   
9  [goats, pasture, grazing, goatherd, herdsman, ...   

                                 Representative_Docs  
0  [Now I shall tell of the city of Zenobia, whic...  
1  [Having said this, I do not wish your eyes to ...  
2  [But all the trades and professions of the liv...  
3  [Ever since the first time I have lingered to ...  
4  [When the sky was cleared of condors, they had...  
5  [The astronomers, after each change takes plac...  
6  [He infers this: if existence in all its momen...  
7  [Finally he comes to Isidora, a city where the...  
8  [When you have forded the river, when you have...  
9  [Cities have no name for me: they are places w...

In [76]:
def display_topic_results(number):
    # Simulating the topic_model.get_topic_info() functionality (using hardcoded values for now)
    representative_docs = representative[number+2]
    representative_words = topic_model.get_topic(number+1)

    # For actual implementation, uncomment and adjust the following lines:
    # topic = number + 1
    # representative_docs = topic_model.get_topic_info()['Representative_Docs'].tolist()
    # representative_words = topic_model.get_topic_info()['Representation'].tolist()
    
    # Create the HTML content
    html_content = f"""
    <div style="border: 1px solid #e0e0e0; padding: 20px; width: 950; font-family: Arial, sans-serif; margin: 0 auto;">
        <h2>Cluster {number+2}</h2>
        <h3>Representation Words</h3>
        <ul>
            {''.join([f'<li>{word}</li>' for word in representative_words])}
        </ul>

        <h3>Top3 Representative Docs</h3>
        <ul>
            {''.join([f'<li>{doc}</li>' for doc in representative_docs])}
        </ul>
    </div>
    """

    # Display the HTML content in the notebook
    display(HTML(html_content))




In [13]:
#get representation words and docs
number=8
topic=number+1

representative=topic_model.get_topic_info()['Representative_Docs'].tolist()
representative_word=topic_model.get_topic_info()['Representation'].tolist()

word=representative_word[topic]
doc=representative[topic]

print ('Representation words are', word)
print('')
print ('Top3 Representative Docs are', doc)

Representation words are ['goats', 'pasture', 'grazing', 'goatherd', 'herdsman', 'meadow', 'streets', 'lands', 'flock', 'grass']

Top3 Representative Docs are ['Cities have no name for me: they are places without leaves, separating one pasture from another, and where the goats are frightened at street corners and scatter.', 'Ask me the names of the grazing lands: I know them all, the Meadow between the Cliffs, the Green Slope, the Shadowed Grass.', "'We have been wandering through its streets, my goats and I, for an age, and we cannot find our way out...' I recognized him, despite his long white beard; it was the same herdsman of long before."]


In [85]:
#get html render result
display_topic_results(-1)

In [14]:
#create label for topics
labels={-1:'Complexity', 0:'Opposite', 1: 'Escape', 2:'Gaze',3:'Disaster',4:'The Fatal Conceit', 5:'Existentialism',6:'Desire', 7:'Unname',8:'Unname'}



In [15]:
topic_model.set_topic_labels(labels)

In [16]:
topic_model.get_topic_info()

Topic  Count                                            Name  \
0     -1    293               -1_outskirts_houses_streets_place   
1      0     72            0_palaces_construction_floors_street   
2      1     46     1_necropolis_corpses_inhabitant_generations   
3      2     44              2_curtain_caravan_camel_glittering   
4      3     30                3_rodents_plagues_rats_invasions   
5      4     29  4_astronomers_constellations_nebula_telescopes   
6      5     25    5_realizing_language_existence_understanding   
7      6     22      6_staircases_travellers_streets_encounters   
8      7     18            7_terraces_fountains_roofs_windmills   
9      8     12                8_goats_pasture_grazing_goatherd   

          CustomName                                     Representation  \
0         Complexity  [outskirts, houses, streets, place, canals, me...   
1           Opposite  [palaces, construction, floors, street, justic...   
2             Escape  [necropolis, corpses, inhabitant, generations,...   
3               Gaze  [curtain, caravan, camel, glittering, window, ...   
4           Disaster  [rodents, plagues, rats, invasions, lairs, fau...   
5  The Fatal Conceit  [astronomers, constellations, nebula, telescop...   
6     Existentialism  [realizing, language, existence, understanding...   
7             Desire  [staircases, travellers, streets, encounters, ...   
8             Unname  [terraces, fountains, roofs, windmills, pyrami...   
9             Unname  [goats, pasture, grazing, goatherd, herdsman, ...   

                                 Representative_Docs  
0  [Now I shall tell of the city of Zenobia, whic...  
1  [Having said this, I do not wish your eyes to ...  
2  [But all the trades and professions of the liv...  
3  [Ever since the first time I have lingered to ...  
4  [When the sky was cleared of condors, they had...  
5  [The astronomers, after each change takes plac...  
6  [He infers this: if existence in all its momen...  
7  [Finally he comes to Isidora, a city where the...  
8  [When you have forded the river, when you have...  
9  [Cities have no name for me: they are places w...

In [17]:
#topic merge
topics_to_merge = [[2, 6]]
topic_model.merge_topics(docs, topics_to_merge)


In [22]:
#rename label
labels={-1:'Complexity', 0:'Opposite', 1: 'Gaze', 2:'Escape',3:'Disaster',4:'The Fatal Conceit', 5:'Existentialism', 6:'Unname',7:'Unname'}


In [23]:
topic_model.set_topic_labels(labels)

In [24]:
topic_model.get_topic_info()

Topic  Count                                            Name  \
0     -1    293               -1_outskirts_houses_streets_place   
1      0     72            0_palaces_construction_floors_street   
2      1     66                  1_curtain_caravan_camel_window   
3      2     46     2_necropolis_corpses_inhabitant_generations   
4      3     30                3_rodents_plagues_rats_invasions   
5      4     29  4_astronomers_constellations_nebula_telescopes   
6      5     25    5_realizing_language_existence_understanding   
7      6     18            6_terraces_fountains_roofs_windmills   
8      7     12                7_goats_pasture_grazing_goatherd   

          CustomName                                     Representation  \
0         Complexity  [outskirts, houses, streets, place, canals, me...   
1           Opposite  [palaces, construction, floors, street, justic...   
2               Gaze  [curtain, caravan, camel, window, cranes, gaze...   
3             Escape  [necropolis, corpses, inhabitant, generations,...   
4           Disaster  [rodents, plagues, rats, invasions, lairs, fau...   
5  The Fatal Conceit  [astronomers, constellations, nebula, telescop...   
6     Existentialism  [realizing, language, existence, understanding...   
7             Unname  [terraces, fountains, roofs, windmills, pyrami...   
8             Unname  [goats, pasture, grazing, goatherd, herdsman, ...   

                                 Representative_Docs  
0  [Now I shall tell of the city of Zenobia, whic...  
1  [Having said this, I do not wish your eyes to ...  
2  [Ever since the first time I have lingered to ...  
3  [But all the trades and professions of the liv...  
4  [When the sky was cleared of condors, they had...  
5  [The astronomers, after each change takes plac...  
6  [He infers this: if existence in all its momen...  
7  [When you have forded the river, when you have...  
8  [Cities have no name for me: they are places w...

### create JSON file

In [25]:
topic_assignment_df = pd.DataFrame({'Document': docs, 'Topic': topics})

In [26]:
sentences=topic_assignment_df['Document'].tolist()

In [27]:
topic=topic_assignment_df['Topic'].tolist()

In [28]:
tube=[]
for index,sent in enumerate(sentences):
    tube.append((sent,topic[index])) 

In [29]:
topic_location=pd.read_csv('/Users/huopeiyang/Desktop/thesis/topic_with_location.csv')

In [30]:
topic_dict = topic_location.groupby('Location')['Topic'].apply(lambda x: list(set(x))).to_dict()


In [31]:
topic_dict

{'10THIN CITIES 1': [8, -1],
 '11CITIES & MEMORY 5': [0, -1],
 '12CITIES & DESIRE 4': [0, 3, -1],
 '13CITIES & SIGNS 3': [0, 8, -1, 1],
 '14THIN CITIES 2': [0, -1],
 '15TRADING CITIES 1': [1, -1],
 '16CITIES & DESIRE 5': [1, 4, -1],
 '17CITIES & SIGNS 4': [8, 1, 3, -1],
 '18THIN CITIES 3': [0, 1, 6, 8, -1],
 '19TRADING CITIES 2': [1, 4, -1],
 '1CITIES & MEMORY 1': [0, -1],
 '20CITIES & EYES 1': [-1],
 '21CITIES & SIGNS 5': [0, 1, -1],
 '22THIN CITIES 4': [0, -1],
 '23TRADING CITIES 3': [0, 2, 3, 7, -1],
 '24CITIES & EYES 2': [1, -1],
 '25CITIES & NAMES 1': [0, 2, -1],
 '26THIN CITIES 5': [0, 1, 3, 8, -1],
 '27TRADING CITIES 4': [-1],
 '28CITIES & EYES 3': [1, 2, -1],
 '29CITIES & NAMES 2': [1, -1, 6],
 '2CITIES & MEMORY 2': [0, 4, -1],
 '30CITIES & THE DEAD 1': [-1],
 '31TRADING CITIES 5': [-1],
 '32CITIES & EYES 4': [0, -1],
 '33CITIES & NAMES 3': [-1],
 '34CITIES & THE DEAD 2': [1, -1],
 '35CITIES & THE SKY 1': [2, -1, 7],
 '36CITIES & EYES 5': [0, -1],
 '37CITIES & NAMES 4': [0, 1, 

In [32]:
chapter_name=pd.read_csv('/Users/huopeiyang/Desktop/thesis/book_score.csv')

In [33]:
chapter=chapter_name['chapter'].tolist()

In [34]:
js_data = []
seen_relationships = set()

for c in chapter:
    dic = {}
    same = []
    dic['name'] = c
    dic['size'] = 10
    for topic in topic_dict[c]:
        if topic not in [-1, 0]:
            for c2 in chapter:
                if c2 != c and (c, c2) not in seen_relationships:
                    if topic in topic_dict[c2]:
                        dic_share = {"target": c2, "topic": topic}
                        same.append(dic_share)
                        seen_relationships.add((c, c2))
                        seen_relationships.add((c2, c))  # to avoid reverse relationships too
    dic['imports'] = same
    js_data.append(dic)


In [35]:
js_data

[{'name': '1CITIES & MEMORY 1', 'size': 10, 'imports': []},
 {'name': '2CITIES & MEMORY 2',
  'size': 10,
  'imports': [{'target': '4CITIES & MEMORY 3', 'topic': 4},
   {'target': '7CITIES & MEMORY 4', 'topic': 4},
   {'target': '9CITIES & SIGNS 2', 'topic': 4},
   {'target': '16CITIES & DESIRE 5', 'topic': 4},
   {'target': '19TRADING CITIES 2', 'topic': 4},
   {'target': '49HIDDEN CITIES 2', 'topic': 4}]},
 {'name': '3CITIES & DESIRE 1', 'size': 10, 'imports': []},
 {'name': '4CITIES & MEMORY 3',
  'size': 10,
  'imports': [{'target': '7CITIES & MEMORY 4', 'topic': 4},
   {'target': '9CITIES & SIGNS 2', 'topic': 4},
   {'target': '16CITIES & DESIRE 5', 'topic': 4},
   {'target': '19TRADING CITIES 2', 'topic': 4},
   {'target': '49HIDDEN CITIES 2', 'topic': 4}]},
 {'name': '5CITIES & DESIRE 2', 'size': 10, 'imports': []},
 {'name': '6CITIES & SIGNS 1',
  'size': 10,
  'imports': [{'target': '8CITIES & DESIRE 3', 'topic': 1},
   {'target': '13CITIES & SIGNS 3', 'topic': 1},
   {'target

In [36]:
import json
with open('book_topic_data.json', 'w') as f:
    json.dump(js_data, f)

In [45]:
#get reference from D3
from urllib.request import urlopen
  
# import json
import json
# store the URL in url as 
# parameter for urlopen
url = "https://raw.githubusercontent.com/holtzy/D3-graph-gallery/master/DATA/data_hierarchical_edge_bundling.json"
  
# store the response of URL
response = urlopen(url)
  
# storing the JSON response 
# from url in data
data_json = json.loads(response.read())
  
# print the json response
print(data_json)

[{'name': 'company.salesManager.salesPersonA', 'size': 12, 'imports': ['company.salesManager.salesPersonB', 'company.HRManager.recruiterB', 'company.engineerManager.engineerB']}, {'name': 'company.salesManager.salesPersonB', 'size': 2, 'imports': []}, {'name': 'company.HRManager.recruiterA', 'size': 120, 'imports': ['company.HRManager.recruiterB', 'company.salesManager.salesPersonA']}, {'name': 'company.HRManager.recruiterB', 'size': 19, 'imports': []}, {'name': 'company.engineerManager.engineerA', 'size': 121, 'imports': []}, {'name': 'company.engineerManager.engineerB', 'size': 127, 'imports': ['company.engineerManager.engineerA']}]


In [46]:
data_json

[{'name': 'company.salesManager.salesPersonA',
  'size': 12,
  'imports': ['company.salesManager.salesPersonB',
   'company.HRManager.recruiterB',
   'company.engineerManager.engineerB']},
 {'name': 'company.salesManager.salesPersonB', 'size': 2, 'imports': []},
 {'name': 'company.HRManager.recruiterA',
  'size': 120,
  'imports': ['company.HRManager.recruiterB',
   'company.salesManager.salesPersonA']},
 {'name': 'company.HRManager.recruiterB', 'size': 19, 'imports': []},
 {'name': 'company.engineerManager.engineerA', 'size': 121, 'imports': []},
 {'name': 'company.engineerManager.engineerB',
  'size': 127,
  'imports': ['company.engineerManager.engineerA']}]